In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import cv2
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import random
import joblib

In [ ]:
# Caminho das imagens e classificações
images_path = 'dataset_converted'

# DataFrame inicial
df = pd.DataFrame(columns=['path', 'classification'])
classification_label = []

In [ ]:
# Carregar caminhos de imagens e classificações
for f_number, folder in enumerate(os.listdir(images_path)):
    folder_path = os.path.join(images_path, folder)
    classification_label.append(folder)
    for image in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image)
        df.loc[0 if pd.isnull(df.index.max()) else df.index.max() + 1] = [image_path, f_number]

In [ ]:
# Verificar se o DataFrame não está vazio
if df.empty:
    raise ValueError("O DataFrame está vazio. Verifique o carregamento das imagens.")

In [ ]:
# Função para carregar e pré-processar a imagem
def preprocess_image(image_path):
    # Carregar a imagem e redimensionar para o tamanho esperado pela ResNet50
    image = np.array(Image.open(image_path).convert('RGB').resize((224, 224)))
    # Pré-processar a imagem de acordo com o ResNet50
    image = preprocess_input(image)
    return image

In [ ]:
# Aplicar o pré-processamento a todas as imagens
df['image'] = df['path'].apply(preprocess_image)

In [ ]:
# Verificar se as imagens foram processadas corretamente
if df['image'].isnull().any():
    raise ValueError("Existem imagens não processadas. Verifique o pré-processamento.")

In [ ]:
# Converter a coluna 'image' para uma lista de arrays
X = np.array(df['image'].tolist(), dtype=np.float32)

In [ ]:
# Verificar se as classificações estão corretas
if df['classification'].isnull().any():
    raise ValueError("Existem classificações faltantes. Verifique os dados de entrada.")

In [ ]:
# Converter classificações para one-hot encoding
y = to_categorical(df['classification'], num_classes=len(classification_label))

In [ ]:
# Verificação adicional
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")
print(f"Data type of X: {X.dtype}")
print(f"Data type of y: {y.dtype}")

In [ ]:
# Separar os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Construir o modelo ResNet50
base_model = ResNet50(weights='imagenet', include_top=False)

In [ ]:
# Adicionar camadas personalizadas ao modelo
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(classification_label), activation='softmax')(x)

In [ ]:
# Definir o modelo completo
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Congelar as camadas da base_model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Compilar o modelo
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo resnet50

In [ ]:
# Treinar o modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# Avaliar o modelo no conjunto de teste
accuracy = model.evaluate(X_test, y_test)
print(f"Acurácia no conjunto de teste: {accuracy[1]:.2f}")

In [ ]:
# Salvar o modelo treinado em um arquivo
model.save('modelo_resnet50.h5')
model.save_weights('pesos_resnet50.weights.h5')

In [ ]:
# Carregar o modelo treinado a partir do arquivo
modelo_carregado = tf.keras.models.load_model('modelo_resnet50.h5')


In [ ]:
# Avaliar o modelo carregado no conjunto de teste
accuracy = modelo_carregado.evaluate(X_test, y_test)
print(f"Acurácia no conjunto de teste após carregar: {accuracy[1]:.2f}")